In [1]:
import pandas as pd
import sys, os

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from crimebb import *

In [3]:
from tqdm.notebook import tqdm

tqdm.pandas()

In [4]:
YEAR="2019"

In [5]:
DATA_PATH="../../data/"
CSV_PATH = f"{DATA_PATH}csv/{YEAR}/summary/"
CSV_PROCESSED = f"{DATA_PATH}csv/{YEAR}/processed/"

In [6]:
verifyDir(CSV_PROCESSED)

### Loading data

In [7]:
crimeBB_data = CrimeBBManager(DATA_PATH, YEAR)

#### website

In [8]:
crimeBB_data.read_sites_2019()
crimeBB_data.website_df

,site_id,site_name
1,0,hackforums.net
2,1,www.kernelmode.info/forum
4,3,offensivecommunity.net/index.php
3,4,www.mpgh.net
5,5,stresserforums.net/index.php
0,6,greysec.net/index.php


#### boards

In [9]:
crimeBB_data.read_boards_2019()
crimeBB_data.boards_df

,board_id,site_id,site_name,board_title,board_url
0,9,6,greysec.net/index.php,Web Application Security,https://greysec.net/forumdisplay.php?fid=9&page=2
1,3,6,greysec.net/index.php,Changelog,https://greysec.net/forumdisplay.php?fid=3&page=2
2,8,6,greysec.net/index.php,General Hacking and Security,https://greysec.net/forumdisplay.php?fid=8&page=2
3,28,6,greysec.net/index.php,Other Languages,https://greysec.net/forumdisplay.php?fid=28
4,5,6,greysec.net/index.php,Miscellaneous Discussion,https://greysec.net/forumdisplay.php?fid=5&page=2
...,...,...,...,...,...
1048,12,5,stresserforums.net/index.php,Server Stress Testing,https://stresserforums.net/forumdisplay.php?fi...
1049,22,5,stresserforums.net/index.php,Contests & Rewards,https://stresserforums.net/forumdisplay.php?fi...
1050,4,5,stresserforums.net/index.php,DDoS Talk & News,https://stresserforums.net/forumdisplay.php?fi...
1051,13,5,stresserforums.net/index.php,DDoS Protection,https://stresserforums.net/forumdisplay.php?fi...


#### Threads

In [ ]:
crimeBB_data.read_threads_2019()
crimeBB_data.threads_df

#### posts

In [ ]:
crimeBB_data.read_posts()
crimeBB_data.posts_df

In [ ]:
os.exit()

#### posts x threads x boards x website

In [ ]:
crimeBB_data.generates()

In [ ]:
crimeBB_data.crimebb_df.isnull().any()

In [ ]:
crimeBB_data.crimebb_df

In [ ]:
os.exit()

#### Detecting languages

In [ ]:
language_to_eval = ["russian", "german", "english"]

In [ ]:
if verifyFile(f"{CSV_PROCESSED}content_languages.csv"):
    content_df = pd.read_csv(f"{CSV_PROCESSED}content_languages.csv", sep="\t", low_memory=False)
    list_post = content_df[~content_df["language_detected"].isna()]["post_id"].values
else:
    content_df = posts_threads_df[["post_id", "content"]].copy()
    # content_df["content_processed"] = content_df["content"].apply(lambda x: re.sub('[\\\\\'\"+@_!#$%^&*,;().<>?/\|\[\]}{~:=\n]', " ", x) )
    # content_df["content_processed"] = content_df["content_processed"].apply(str.lower)
    content_df["lang_ratio"] = np.nan
    content_df["language_detected"] = np.nan
    content_df["lang_correct_words"] = np.nan
    content_df["len_correct_words"] = np.nan
    content_df["lang_incorrect_words"] = np.nan
    content_df["len_incorrect_words"] = np.nan
    list_post = []

In [ ]:
os.exit()

In [ ]:
count = 1
for index, row in tqdm(content_df.iterrows()):
    
    lang_ratio, lang_badwords, lang_goodwords = detect_language_and_words(row["content"], language_to_eval=language_to_eval)
    posts_threads_df.loc[index, "lang_ratio"] = [lang_ratio]
    lang_detected = max(lang_ratio, key=lang_ratio.get)
    content_df.loc[(d_manejo_key['TpRem'] == 'MANEJO'), "language_detected"] = lang_detected
    content_df.loc[index, "lang_correct_words"] = [{lang_detected: lang_goodwords[lang_detected]}]
    content_df.loc[index, "len_correct_words"] = len(set(lang_goodwords[lang_detected]))
    content_df.loc[index, "lang_incorrect_words"] = [{lang_detected: lang_badwords[lang_detected]}]
    content_df.loc[index, "len_incorrect_words"] = len(set(lang_badwords[lang_detected]))

    if count%10000==0: # grava cada 10 k muestras
        content_df.to_csv(f"{CSV_PROCESSED}content_languages.csv", sep='\t', index=False)
    
    count+=1
    

In [ ]:
content_df.to_csv(f"{CSV_PROCESSED}content_languages.csv", sep='\t', index=False)

### Languages

In [ ]:
lang_post_df = content_df.pivot_table(columns=['language_detected'], aggfunc='size')
lang_post_df.sort_values(ascending=False, inplace=True)
lang_post_df

In [ ]:
mean_val = round(lang_post_df.mean(), 2)

import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(16,8))
ax.bar(lang_post_df.index.values, lang_post_df.values)

for x,y,p in zip(lang_post_df.index.values, lang_post_df.values, lang_post_df.values):
  plt.text(x, y, p)
plt.xlabel("Languages detected")
plt.ylabel("Number of documents")
plt.title(f"Number of documents per language, Avg: {mean_val}")
plt.show()

#### Russian

In [ ]:
# russian_df = content_df[content_df["language_detected"]=="russian"].copy()
# russian_df.info(), russian_df.shape, russian_df.iloc[0,:]

In [ ]:
# russian_content = ' '.join(russian_df["content"])

In [ ]:
# russian_dict = get_text_frequency(russian_content)
# len(russian_dict)

In [ ]:
# showWordCloud(russian_dict, f"Russian-Wordcloud", max_words=int(len(russian_dict)))

#### English

In [ ]:
# english_df = content_df[content_df["language_detected"]=="english"].copy()
# english_df.info(), english_df.shape, english_df.iloc[0,:]

In [ ]:
# english_content = ' '.join(english_df["content"])

In [ ]:
# english_dict = get_text_frequency(english_content)
# len(english_dict)

In [ ]:
# showWordCloud(english_dict, f"English-Wordcloud", max_words=int(len(english_dict)))

#### German

In [ ]:
# german_df = content_df[content_df["language_detected"]=="german"].copy()
# german_df.info(), german_df.shape, german_df.iloc[0,:]

In [ ]:
# german_content = ' '.join(german_df["content"])

In [ ]:
# german_dict = get_text_frequency(german_content)
# len(german_dict)

In [ ]:
# showWordCloud(german_dict, f"German-Wordcloud", max_words=int(len(german_dict)))